In [ ]:
!pip install dask[dataframe] --upgrade

Only needed for Google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%matplotlib inline
import xml.etree.ElementTree as et
import pandas as pd
import dask.dataframe as dd
import numpy as np
import matplotlib.pyplot as plt
import time, datetime

Set pathes for ``Export.xml`` and ``df.csv``. **These must be adjusted!**





In [ ]:
export_xml_file_path = '/content/drive/My Drive/explore_ios_health/apple_health_export/Export.xml'
apple_health_csv_file_path = '/content/drive/My Drive/explore_ios_health/apple_health_export/df.csv'

In [ ]:
xtree = et.parse(export_xml_file_path)
xroot = xtree.getroot()

In [ ]:
def rows_columns():
  
  types = set([])
  rows = []
    
  for node in xroot[0:]: 

    types.add(node.attrib.get('type'))
    
    if node.attrib.get('type') == 'HKQuantityTypeIdentifierHeartRate' or \
      node.attrib.get('type') == 'HKQuantityTypeIdentifierActiveEnergyBurned' or \
      node.attrib.get('type') == 'HKQuantityTypeIdentifierDistanceWalkingRunning' or \
      node.attrib.get('type') == 'HKQuantityTypeIdentifierAppleExerciseTime' or \
      node.attrib.get('type') == 'HKQuantityTypeIdentifierAppleStandTime' or \
      node.attrib.get('type') == 'HKQuantityTypeIdentifierStepCount':    

        s_type = node.attrib.get('type')
        s_unit = node.get('unit')
        s_date = node.get('startDate')
        s_value = float(node.get('value'))
        s_source = node.get('sourceName')

        columns = ['type','source', 'unit', 'dateTime', 'value']
        
        rows.append({'type': s_type, 
                     'unit': s_unit, 
                     'dateTime': s_date,
                     'source': s_source, 
                     'value': s_value})


  print('Available data types:')
  for data_type in types:
    print(data_type) 
  return (rows, columns) 

In [ ]:
rows, columns = rows_columns()
df = pd.DataFrame(rows, columns=columns)
df = df.dropna()

print(f'\nShape:{df.shape}')
df.head()

Remove objects to free some space.

In [ ]:
del(rows)
del(columns)
del(xtree)
del(xroot)

Store data frame in CSV if needed.

In [ ]:
df.to_csv(apple_health_csv_file_path, index=False)

Load data frame from CSV if needed.

In [ ]:
df = pd.read_csv(apple_health_csv_file_path)

print(f'Shape:{df.shape}')
df.head()

Two different merge functions used for putting the final data frame together.

In [ ]:
def merge_date_formated_df(df, data_type, column_name, grouped_strategy):
    
    # Get all rows from `data_type`
    grouped_df = df[df['type'].isin([data_type])]
    grouped_df = grouped_df.drop({'type','source','unit'}, axis=1)
    grouped_df = grouped_df.rename(columns={'value': column_name})

    # Floor `date_time` to minutes
    date_time = pd.to_datetime(grouped_df['dateTime'])
    date_time = date_time.dt.tz_localize(None)
    grouped_df['dateTime'] = date_time.dt.floor('min').copy()

    # Group by `date_time` and set value to mean or sum
    if grouped_strategy == 'Mean':
        grouped_df = round(grouped_df.groupby('dateTime', as_index=False).mean(), 0)
    elif grouped_strategy == 'Sum':
        grouped_df = round(grouped_df.groupby('dateTime', as_index=False).sum(), 4)
    else:
        raise ValueError(
            "Invalid value for 'grouped_strategy'! Should be either 'mean' or 'sum'.")
    
    # Create new `date` column
    date = pd.to_datetime(grouped_df['dateTime'])
    date = date.dt.tz_localize(None)
    grouped_df['date'] = date.dt.floor('d').copy()

    # Use new `date` column to create new columns containing the average or the sum
    if grouped_strategy == 'Mean':
      new_df = pd.DataFrame(grouped_df.groupby('date', as_index=False)[column_name].mean())
    elif grouped_strategy == 'Sum':
      new_df = pd.DataFrame(grouped_df.groupby('date', as_index=False)[column_name].sum())

    new_df = new_df.rename(columns={column_name: f'{column_name}{grouped_strategy}'})
    merged = pd.merge(grouped_df, new_df, on='date', how='outer')
    merged = merged.set_index('dateTime')
    return merged

def merge(dataframes_list):
    if len(dataframes_list) < 2:
        raise ValueError(
            "'dataframes_list' should contain at least two dataframes!")

    merged = dataframes_list[0].merge(dataframes_list[1], how='outer',
                      left_index=True, right_index=True)

    if len(dataframes_list) > 2:
        for df in dataframes_list[2:]:
            merged = merged.merge(df, how='outer',
                              left_index=True, right_index=True)
    return merged

Store data frames to CSV to handle them with desk.

In [ ]:
merge_date_formated_df(
    df, 'HKQuantityTypeIdentifierHeartRate', 
    'heartRate(BPM)', 
    'Mean').to_csv("hr_temp.csv")

merge_date_formated_df(
    df, 
    'HKQuantityTypeIdentifierDistanceWalkingRunning', 
    'walkingRunningDistance(km)', 
    'Sum').to_csv('wd_temp.csv',index_label='dateTime')

merge_date_formated_df(
    df, 
    'HKQuantityTypeIdentifierActiveEnergyBurned', 
    'activeEnergyBurned(kal)', 
    'Sum').to_csv('aeb_temp.csv',index_label='dateTime')

merge_date_formated_df(
    df, 
    'HKQuantityTypeIdentifierAppleStandTime', 
    'standTime(min)', 
    'Sum').to_csv('st_temp.csv',index_label='dateTime')

merge_date_formated_df(
    df, 
    'HKQuantityTypeIdentifierStepCount', 
    'stepCount', 'Sum').to_csv('sc_temp.csv',index_label='dateTime')

Load data frames from using [Desk](https://dask.org/). This is needed so it can handle big data frame.

In [ ]:
hr = dd.read_csv('hr_temp.csv').set_index('dateTime')
wd = dd.read_csv('wd_temp.csv').set_index('dateTime')
aeb = dd.read_csv('aeb_temp.csv').set_index('dateTime')
st = dd.read_csv('st_temp.csv').set_index('dateTime')
sc = dd.read_csv('sc_temp.csv').set_index('dateTime')

Merge the different data frames together and handle NaNs.

In [ ]:
df_list = [df.drop(columns={'date'}) for df in [hr, wd, aeb, st, sc]]

merged = merge(df_list)

merged['standTime(min)'] = merged['standTime(min)'].fillna(0.0)
merged['standTime(min)Sum'] = merged['standTime(min)Sum'].fillna(0.0)
merged['stepCount'] = merged['stepCount'].fillna(0)
merged['stepCountSum'] = merged['stepCountSum'].fillna(0)
merged['activeEnergyBurned(kal)'] = merged['activeEnergyBurned(kal)'].fillna(0.0)
merged['activeEnergyBurned(kal)Sum'] = merged['activeEnergyBurned(kal)Sum'].fillna(0.0)
merged['walkingRunningDistance(km)'] = merged['walkingRunningDistance(km)'].fillna(0.0)
merged = merged.dropna()
print(type(merged))

Remove objects to free some space.

In [ ]:
del(hr)
del(wd)
del(aeb)
del(st)
del(sc)

For split dateTime, rouning numbers and change order.

In [ ]:
def formated_df(df):
  df = df.round({
    'heartRate(BPM)':1, 
    'heartRate(BPM)Mean':1, 
    'walkingRunningDistance(km)':4,
    'walkingRunningDistance(km)Sum':4,
    'activeEnergyBurned(kal)':1,
    'activeEnergyBurned(kal)Sum':1,
    'standTime(min)':1,
    'standTime(min)Sum':1,
    'stepCount':1,
    'stepCountSum':1,
    })
  df.index = pd.to_datetime(df.index)
  df['date'] = df.index.date
  df['time'] = df.index.time
  df['weekday'] = df.index.weekday

  df = df[['date', 
          'time', 
          'weekday',
          'heartRate(BPM)', 
          'heartRate(BPM)Mean', 
          'walkingRunningDistance(km)', 
          'walkingRunningDistance(km)Sum',
          'activeEnergyBurned(kal)',
          'activeEnergyBurned(kal)Sum',
          'standTime(min)',
          'standTime(min)Sum',
          'stepCount',
          'stepCountSum'
          ]]

  return df
  

Compute pandas dataframe from dask dataframe.

In [ ]:
health_df = formated_df(merged.compute())

print(f'Shape:{df.shape}')
print(type(health_df))
print(f'\nMax values:')
print(health_df.max())
health_df.head()

Remove objects to free some space.

In [ ]:
del(merged)

In [ ]:
plt.plot('date','heartRate(BPM)Mean', data=health_df)
plt.show()
plt.plot('date','standTime(min)Sum', data=health_df)
plt.show()
plt.plot('date','walkingRunningDistance(km)Sum', data=health_df)
plt.show()
plt.plot('date','activeEnergyBurned(kal)Sum', data=health_df)
plt.show()
plt.plot('date','stepCountSum', data=health_df)
plt.show()

In [ ]:
data = pd.DataFrame(health_df.groupby(['weekday'])['heartRate(BPM)Mean'].mean())
data = data['heartRate(BPM)Mean']
labels = data.index

plt.xticks(labels, ['Mon', 'The', 'Wed', 'Thu','Fri', 'Sat', 'Sun'])
plt.xlabel('Weekday')
plt.ylabel('Average heart rate (BPM)')
plt.bar(labels, data)
plt.show()

In [ ]:
plt.scatter('heartRate(BPM)','activeEnergyBurned(kal)', data=health_df)

plt.show()